In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
root = '/kaggle/input/titanic/'
#Any results you write to the current directory are saved as output.

/kaggle/input/titanic-result/titanic.csv
/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv


In [2]:
import pandas as pd
#train_dict_df = {'PassengerId' : 'int8','Pclass': 'int8', 'Age':'int8','Fare':'float32', 'Cabin':'int8' }
train_df = pd.read_csv(
    os.path.join(root, 'train.csv'))
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


MODIFY THE AGE DATA

In [3]:
import numpy as np 
for index, row in train_df.iterrows():
    if np.isnan(row['Age']):
        train_df.at[index, 'Age'] = round(train_df['Age'].mean())
train_df['Sex'] = pd.Categorical(train_df['Sex']).codes 
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",1,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",0,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",0,30.0,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",1,26.0,0,0,111369,30.0000,C148,C


Data engineering 

In [4]:

train_df_reg = train_df.drop(labels = ['PassengerId', 'Name', 'SibSp', 'Ticket', 'Cabin', 'Embarked', 'Parch'], axis = 1)
corr = train_df.corr(method = 'pearson')
survived = train_df_reg.pop('Survived')
corr

train_df_reg.describe()

,Pclass,Sex,Age,Fare
count,891.000000,891.000000,891.000000,891.000000
mean,2.308642,0.647587,29.758889,32.204208
std,0.836071,0.477990,13.002570,49.693429
min,1.000000,0.000000,0.420000,0.000000
25%,2.000000,0.000000,22.000000,7.910400
50%,3.000000,1.000000,30.000000,14.454200
75%,3.000000,1.000000,35.000000,31.000000
max,3.000000,1.000000,80.000000,512.329200


Simple Linear regression 

In [5]:
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
scaler = MinMaxScaler()


clf_train = scaler.fit_transform(train_df_reg)
linreg = SVC(kernel='rbf', gamma = 'scale', C=20)
linreg_pred = linreg.fit(clf_train, survived)
#linreg = Ridge(alpha = 30).fit(clf_train, survived)
scores = cross_val_score(linreg, clf_train, survived,  cv=3)
scores

array([0.77441077, 0.81481481, 0.79124579])

Importing test data

In [6]:
import pandas as pd
#train_dict_df = {'PassengerId' : 'int8','Pclass': 'int8', 'Age':'int8','Fare':'float32', 'Cabin':'int8' }
test_df = pd.read_csv(
    os.path.join(root, 'test.csv'))
test_df['Sex'] = pd.Categorical(test_df['Sex']).codes 
test_df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",1,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",1,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",1,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",1,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",0,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",1,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",1,NaN,0,0,359309,8.0500,NaN,S


Filling missing value in test data

In [7]:
import numpy as np 
for index, row in test_df.iterrows():
    if np.isnan(row['Age']):
        test_df.at[index, 'Age'] = round(test_df['Age'].mean())
test_df.ffill(inplace=True)
test_df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",1,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",1,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",1,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",1,30.0,0,0,A.5. 3236,8.0500,C78,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",0,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",1,38.5,0,0,SOTON/O.Q. 3101262,7.2500,C105,S
416,1308,3,"Ware, Mr. Frederick",1,30.0,0,0,359309,8.0500,C105,S


Fill the dataframe with the predicted values

In [8]:

test_df_reg = test_df.drop(labels = ['PassengerId', 'Name', 'SibSp', 'Ticket', 'Cabin', 'Embarked', 'Parch'], axis = 1)

clf_test = scaler.transform(test_df_reg)

test_re = linreg_pred.predict(clf_test)
test_df['Survived'] = np.round_(test_re)



Export to the CSV file

In [9]:
res = pd.DataFrame(test_df, columns = ['PassengerId', 'Survived'])
res.to_csv('titanic.csv',index=False)
